In [ ]:
%scala
val storage_container = "data"
val storage_account_name = "<NAME>"
val storage_account_access_key = "<KEY>"
val storage_base_url = s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net"

val file_type = "parquet"
val train_data_url = s"${storage_base_url}/data/train/train_data/*.${file_type}"
val  test_data_url = s"${storage_base_url}/data/test/test_data/*.${file_type}"

storage_container: String = data
storage_account_name: String = cs1100320004e0010e6
storage_account_access_key: String = an5UdS00UuMcb6vT54s2kWCF31rW++P9BKK7K0czqJDmONenltWmfatssB+4tAQOAHbYOUALp2RB+AStHj88Gg==
storage_base_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net
file_type: String = parquet
train_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/train/train_data/*.parquet
test_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/test/test_data/*.parquet

In [ ]:
%scala
import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType}
import org.apache.spark.sql.SparkSession

val spark = ( SparkSession.builder()
                          .appName("Pre-Processing-2")
                          .config(s"fs.azure.account.key.${storage_account_name}.blob.core.windows.net", storage_account_access_key)
                          .getOrCreate() )
val sc = spark.sparkContext

import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType}
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3b6e3fe
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2cb4dbcf

In [ ]:
%scala
val stopWordsEN = Array(
 "i","me","my","myself",
 "we","our","ours","ourselves",
 "you","you're","you've","you'll","you'd","your","yours","yourself","yourselves",
 "he","him","his","himself",
 "she","she's","her","hers","herself",
 "it","it's","its","itself",
 "they","them","their","theirs","themselves",
 "what","which","who","whom",
 "this","that","that'll","these","those",
 "am","is","are","was","were","be","been","being",
 "have","has","had","having",
 "do","does","did","doing",
 "a","an","the",
 "of","at","by","for","with",
 "and", "as",
 "about","against","between","into","through","during",
 "before","after",
 "above","below",
 "to","from","up","down","in","out","on","off","over","under",
 "again","further","then","once","here","there",
 "when","where","why","how","all","any","both","each",
 "s","t",
 "can","will","just",
 "should","should've",
 "now","d","ll","m","o","re","ve","y","ma"
)

stopWordsEN: Array[String] = Array(i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, s, t, can, will, just, should, should've, now, d, ll, m, o, re, ve, y, ma)

In [ ]:
%scala
import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, PerceptronModel, StopWordsCleaner}
import com.johnsnowlabs.nlp.base.DocumentAssembler
import org.apache.spark.ml.Pipeline
import spark.implicits._

val documentAssembler = new DocumentAssembler()
  .setInputCol("comment")
  .setOutputCol("document")

val documentNormalizer = new DocumentNormalizer()
  .setInputCols("document")
  .setOutputCol("normalizedDocument")
  .setAction("clean")
  .setPatterns(Array("<[^>]>", """[^\w\d\s]"""))
  .setReplacement(" ")
  .setPolicy("pretty_all")
  .setLowercase(true)

val tokenizer = new Tokenizer()
  .setInputCols(Array("normalizedDocument"))
  .setOutputCol("token")

val stopWords = new StopWordsCleaner()
  .setInputCols("token")
  .setOutputCol("cleanTokens")
  .setStopWords(stopWordsEN)

val lemmatizer = LemmatizerModel.pretrained()
  .setInputCols(Array("cleanTokens"))
  .setOutputCol("lemma")

val posTagger = PerceptronModel.pretrained("pos_ud_ewt", "en")
  .setInputCols(Array("normalizedDocument", "cleanTokens"))
  .setOutputCol("posTag")

val pipeline = new Pipeline()
  .setStages(Array(
    documentAssembler,
    documentNormalizer,
    tokenizer,
    stopWords,
    lemmatizer,
    posTagger
  ))

import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, PerceptronModel, StopWordsCleaner}
import com.johnsnowlabs.nlp.base.DocumentAssembler
import org.apache.spark.ml.Pipeline
import spark.implicits._
documentAssembler: com.johnsnowlabs.nlp.DocumentAssembler = document_d2c30105b2bc
documentNormalizer: com.johnsnowlabs.nlp.annotators.DocumentNormalizer = DOCUMENT_NORMALIZER_a497a10a3c99
tokenizer: com.johnsnowlabs.nlp.annotators.Tokenizer = REGEX_TOKENIZER_377de1f4aa75
stopWords: com.johnsnowlabs.nlp.annotators.StopWordsCleaner = STOPWORDS_CLEANER_c0f05b3a9408
lemmatizer: com.johnsnowlabs.nlp.annotators.LemmatizerModel = LEMMATIZER_c62ad8f355f9
posTagger: com.johnsnowlabs.nlp.annotators.pos.perceptron.PerceptronModel = POS_3ceadb13f4a7
pipeline: org.apache.spark.ml.Pipeline = pipeline_25e3bbe8770a

In [ ]:
%scala
val schema = StructType(
  StructField("id"     , IntegerType, true) ::
  StructField("rating" , IntegerType, true) ::
  StructField("type"   , IntegerType, true) ::
  StructField("comment", StringType , true) :: Nil
)

val trainDF = spark.read.format(file_type)
                        .schema(schema)
                        .load(train_data_url)

val testDF  = spark.read.format(file_type)
                        .schema(schema)
                        .load(test_data_url)

val transformedTrainDF = pipeline.fit(trainDF)
                                 .transform(trainDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("lemma")("result").as("comment"),
                                    col("posTag")("result").as("posTag")
                                  )
                                 .cache()

val transformedTestDF  = pipeline.fit(testDF)
                                 .transform(testDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("lemma")("result").as("comment"),
                                    col("posTag")("result").as("posTag")
                                  )
                                 .cache()

schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,true),StructField(rating,IntegerType,true),StructField(type,IntegerType,true),StructField(comment,StringType,true))
trainDF: org.apache.spark.sql.DataFrame = [id: int, rating: int ... 2 more fields]
testDF: org.apache.spark.sql.DataFrame = [id: int, rating: int ... 2 more fields]
transformedTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 3 more fields]
transformedTestDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 3 more fields]

In [ ]:
%scala
transformedTrainDF.show(5, 50)

+-----+------+---------+--------------------------------------------------+--------------------------------------------------+
 id|rating|sentiment| comment| posTag|
+-----+------+---------+--------------------------------------------------+--------------------------------------------------+
16088| 0| -1|[spoiler, spoiler, first, godzilla, movie, thir...|[NOUN, NOUN, ADV, NOUN, NOUN, ADJ, NOUN, NOUN, ...|
 1175| 9| 1|[match, 1, tag, team, table, match, bubba, ray,...|[VERB, NUM, NOUN, NOUN, NOUN, NOUN, NOUN, NOUN,...|
 4383| 9| 1|[attention, spoiler, first, let, say, rob, roy,...|[NOUN, NOUN, ADV, VERB, VERB, VERB, NOUN, NUM, ...|
10044| 9| 1|[titanic, direct, james, cameron, present, fict...|[ADV, VERB, NOUN, NOUN, NOUN, ADJ, NOUN, NOUN, ...|
 4076| 10| 1|[probably, hear, bit, new, disney, dub, miyazak...|[ADV, VERB, NOUN, ADJ, NOUN, NOUN, NOUN, NOUN, ...|
+-----+------+---------+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTestDF.show(5, 50)

+----+------+---------+--------------------------------------------------+--------------------------------------------------+
 id|rating|sentiment| comment| posTag|
+----+------+---------+--------------------------------------------------+--------------------------------------------------+
8647| 10| 1|[sign, lose, highway, say, major, spoiler, ahea...|[VERB, VERB, NOUN, VERB, ADJ, NOUN, ADV, CCONJ,...|
3801| 7| 1|[some, spoiler, include, although, many, commen...|[DET, NOUN, VERB, SCONJ, ADJ, NOUN, VERB, NOUN,...|
4051| 10| 1|[back, mid, late, 80s, oav, anime, title, bubbl...|[ADV, X, ADJ, NOUN, ADJ, ADV, NOUN, NOUN, NOUN,...|
 754| 9| 1|[if, anyone, ever, assemble, compendium, modern...|[SCONJ, PRON, ADV, VERB, ADJ, ADJ, ADJ, NOUN, A...|
3958| 9| 1|[film, belong, film, noir, genre, usually, cont...|[NOUN, VERB, NOUN, NOUN, NOUN, ADV, VERB, ADJ, ...|
+----+------+---------+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTrainDF
       .withColumn("comment", concat_ws("|", col("comment") ).as("comment") )
       .withColumn("posTag" , concat_ws("|", col("posTag") ).as("posTag") )
       .repartition(1)
       .write
       .mode("overwrite")
       .option("header", "true")
       .format("com.databricks.spark.csv")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/transformed_train")

transformedTestDF
       .withColumn("comment", concat_ws("|", col("comment") ).as("comment") )
       .withColumn("posTag" , concat_ws("|", col("posTag") ).as("posTag") )
       .repartition(1)
       .write
       .mode("overwrite")
       .option("header", "true")
       .format("com.databricks.spark.csv")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/transformed_test")

In [ ]:
%scala
transformedTrainDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/transformed_train")

transformedTestDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/transformed_test")